## Data Science BootCamp Week 5 Take Home
#### Chapter 1, Monday sessions
By Yasamin Esmaeili

In [1]:
import pandas as pd

### 1. Actors and Directors Who Cooperated At Least Three Times

[Link to question](https://leetcode.com/problems/actors-and-directors-who-cooperated-at-least-three-times/)

The goal is to filter the dataset based on the actor and director id pairs that appear at least 3 times, each with a unique timestamp (separate films.)

In [2]:
# Example for 'Actors and Directors'
data = {'actor_id': [1, 1, 1, 1, 1, 2, 2],
        'director_id': [1, 1, 1, 2, 2, 1, 1],
        'timestamp': [0, 1, 2, 3, 4, 5, 6]}
ActorDirector = pd.DataFrame(data)
print("Input DataFrame:")
print(ActorDirector)

Input DataFrame:
   actor_id  director_id  timestamp
0         1            1          0
1         1            1          1
2         1            1          2
3         1            2          3
4         1            2          4
5         2            1          5
6         2            1          6


SQL Solution:

In [3]:
# SQL Solution:
sql_query = """
SELECT actor_id, director_id
FROM ActorDirector
GROUP BY 1, 2
HAVING COUNT(DISTINCT timestamp) >= 3;
"""

Pandas Solution:

In [4]:
def actors_and_directors(actor_director: pd.DataFrame) -> pd.DataFrame:
    df = actor_director.groupby(['actor_id', 'director_id']).count().reset_index() # Count timestamps for each actor-director pair
    df = df[df['timestamp']>=3] # Filter pairs with at least 3 unique timestamps
    return df[['actor_id','director_id']].drop_duplicates() # Return only actor_id and director_id columns

In [5]:
print("\nPandas Result:")
print(actors_and_directors(ActorDirector))


Pandas Result:
   actor_id  director_id
0         1            1


### 2. Fix Names in a Table

[Link to question](https://leetcode.com/problems/fix-names-in-a-table/)

The goal is to lowercase the whole string first, then uppercase the first character only. Finally order it by user_id.

In [6]:
# Example for 'Fix Names in a Table'
data = {'user_id': [1, 2],
        'name': ['aLice', 'bOB']}
Users = pd.DataFrame(data)
print("Input DataFrame for Fix Names:")
print(Users)

Input DataFrame for Fix Names:
   user_id   name
0        1  aLice
1        2    bOB


SQL Solution:

In [7]:
# SQL Solution
sql_query_fix_names = """
SELECT user_id, CONCAT(UPPER(LEFT(name,1)),LOWER(SUBSTR(name, 2))) AS name
FROM Users
ORDER BY user_id
"""

Pandas Solution:

In [8]:
def fix_names(users: pd.DataFrame) -> pd.DataFrame:
    users['name'] = users['name'].str.capitalize() # Capitalize first letter and lowercase the rest
    return users.sort_values(by='user_id').reset_index(drop=True) # Order by user_id

In [9]:
print("\nPandas Result:")
print(fix_names(Users))


Pandas Result:
   user_id   name
0        1  Alice
1        2    Bob


### 3. Combine Two Tables

[Link to question](https://leetcode.com/problems/combine-two-tables/)

The goal is to match the two tables based on the personId, adding the Address table from right to the Person table.

In [10]:
# Example for 'Combine Two Tables'
data_person = {'personId': [1, 2],
                'firstName': ['Allen', 'Bob'],
                'lastName': ['Wong', 'Alice']}
Person = pd.DataFrame(data_person)
data_address = {'addressId': [1, 2],
                 'personId': [2, 3],
                 'city': ['New York', 'Leetcode'],
                 'state': ['New York', 'California']}
Address = pd.DataFrame(data_address)
print("Input DataFrame for Person:")
print(Person)
print("Input DataFrame for Address:")
print(Address)

Input DataFrame for Person:
   personId firstName lastName
0         1     Allen     Wong
1         2       Bob    Alice
Input DataFrame for Address:
   addressId  personId      city       state
0          1         2  New York    New York
1          2         3  Leetcode  California


SQL Solution:

In [11]:
# SQL Solution:
sql_query_combine = """
SELECT P.firstName, P.lastName, A.city, A.state
FROM Person AS P
LEFT JOIN Address AS A
ON P.personId = A.personId
"""

Pandas Solution:

In [12]:
def combine_two_tables(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
    return pd.merge(person,address[['personId', 'city', 'state']],how='left',on='personId')[['firstName','lastName','city','state']]

We dropped addressId since it wasn't required. We merge the two data frames based on the personId, keeping the Person data frame on the left, and inserting the matching values from the Address dataframe from the right side.

In [13]:
print("\nPandas Result:")
print(combine_two_tables(Person, Address))


Pandas Result:
  firstName lastName      city     state
0     Allen     Wong       NaN       NaN
1       Bob    Alice  New York  New York


### 4. Second Highest Salary

[Link to Question](https://leetcode.com/problems/second-highest-salary/)

The goal here is to find the second highest distinct salary from the Employee table, and if there's none, return None.

In [14]:
# Example for 'Second Highest Salary'
data_employee = {'id': [1, 2, 3],
                 'salary': [100, 200, 300]}
Employee = pd.DataFrame(data_employee)
print("Input DataFrame for Employee:")
print(Employee)

Input DataFrame for Employee:
   id  salary
0   1     100
1   2     200
2   3     300


SQL Solution:

In [15]:
# SQL Solution:
# Using MAX function and filter 
sql_query_second_highest = """
SELECT MAX(salary) AS SecondHighestSalary
FROM Employee
WHERE Salary < (SELECT MAX(Salary) FROM Employee)
"""

# Using SUBQUERY and LIMIT
sql_query_second_highest_limit = """
SELECT (
    SELECT DISTINCT Salary 
    FROM Employee
    ORDER BY Salary DESC
    LIMIT 1 OFFSET 1
) AS SecondHighestSalary
"""

Pandas Solution:

In [16]:
def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    max_val = -1
    max_val2 = -1

    for val in employee['salary']:
        if (val>max_val2) & (val>max_val):
            max_val2 = max_val
            max_val = val
        elif (val>max_val2) & (val<max_val):
            max_val2 = val

    if max_val2>-1:
        return pd.DataFrame({'SecondHighestSalary': [max_val2]})
    else:
        return pd.DataFrame({'SecondHighestSalary': [None]})

In [17]:
print("\nPandas Result:")
print(second_highest_salary(Employee))


Pandas Result:
   SecondHighestSalary
0                  200


### 5. List the Products Ordered in a Period

[Link to question](https://leetcode.com/problems/list-the-products-ordered-in-a-period/)

The goal is to merge the two tables, and then group them based on their product_id/name, and filter them based on date and sum of the order units.

In [18]:
# Example for 'List the Products Ordered in a Period'
data_products = {'product_id': [1, 2, 3, 4, 5],
                    'product_name': ['Leetcode Solutions', 'Jewels of Stringology', 'HP', 'Lenovo', 'Leetcode Kit'],
                    'product_category': ['Book', 'Book', 'Electronics', 'Electronics', 'T-Shirt']}
Products = pd.DataFrame(data_products)
data_orders = {'product_id': [1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5],
                     'order_date': ['2020-02-05', '2020-02-10', '2020-01-18', '2020-02-11', 
                                    '2020-02-17', '2020-02-24', '2020-03-03', '2020-03-04',
                                    '2020-03-04', '2020-02-25', '2020-02-27', '2020-03-01'],
                        'unit': [60, 70, 30, 80, 2, 3, 20, 30, 60, 50, 50, 50]}
Orders = pd.DataFrame(data_orders)
print("Input DataFrame for Products:")
print(Products)
print("Input DataFrame for Orders:")
print(Orders)

Input DataFrame for Products:
   product_id           product_name product_category
0           1     Leetcode Solutions             Book
1           2  Jewels of Stringology             Book
2           3                     HP      Electronics
3           4                 Lenovo      Electronics
4           5           Leetcode Kit          T-Shirt
Input DataFrame for Orders:
    product_id  order_date  unit
0            1  2020-02-05    60
1            1  2020-02-10    70
2            2  2020-01-18    30
3            2  2020-02-11    80
4            3  2020-02-17     2
5            3  2020-02-24     3
6            4  2020-03-03    20
7            4  2020-03-04    30
8            4  2020-03-04    60
9            5  2020-02-25    50
10           5  2020-02-27    50
11           5  2020-03-01    50


SQL Solution:

In [19]:
# SQL Solution:
sql_query_list_products = """
SELECT P.product_name, SUM(O.unit) AS unit
FROM Orders as O 
INNER JOIN Products AS P
ON O.product_id = P.product_id
WHERE O.order_date BETWEEN "2020-02-01" AND "2020-02-29"
GROUP BY P.product_name
HAVING SUM(O.unit)>=100
"""

Pandas Solution:

In [20]:
def list_products(products: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    orders['order_date'] = pd.to_datetime(orders['order_date']) # Convert order_date to datetime
    #   Merge orders within date range with products on product_id
    merged_df = pd.merge(orders[(orders['order_date'] >= '2020-02-01') & (orders['order_date'] <= '2020-02-29')] , products, on='product_id', how='inner')
    #   Return product_name and sum of units where sum of units >=100
    return merged_df.groupby('product_name', as_index=False)['unit'].sum().query('unit >= 100')[['product_name', 'unit']]

In [21]:
print("\nPandas Result:")
print(list_products(Products, Orders))


Pandas Result:
         product_name  unit
2        Leetcode Kit   100
3  Leetcode Solutions   130


### 6. Replace Employee ID With The Unique Identifier

[Link to question](https://leetcode.com/problems/replace-employee-id-with-the-unique-identifier/)

The goal is to merge the Employee table with the unique identifiers based on their ids.

In [22]:
# Example for 'Replace Employee ID With The Unique Identifier'
data_employees = {'id': [1, 7, 11, 90, 3],
                  'name': ['Alice', 'Bob', 'Meir', 'Winston', 'Jonathan']}
Employees = pd.DataFrame(data_employees)
data_employeeuni = {'id': [3, 11, 90],
                    'unique_id': [1, 2, 3]}
EmployeeUNI = pd.DataFrame(data_employeeuni)
print("Input DataFrame for Employees:")
print(Employees)
print("Input DataFrame for EmployeeUNI:")
print(EmployeeUNI)

Input DataFrame for Employees:
   id      name
0   1     Alice
1   7       Bob
2  11      Meir
3  90   Winston
4   3  Jonathan
Input DataFrame for EmployeeUNI:
   id  unique_id
0   3          1
1  11          2
2  90          3


SQL Solution:

In [23]:
# SQL Solution:
sql_query_replace_employee_id = """
SELECT U.unique_id, E.name
FROM Employees E
LEFT JOIN EmployeeUNI U
ON E.id = U.id
"""

Pandas Solution:

In [24]:
def replace_employee_id(employees: pd.DataFrame, employee_uni: pd.DataFrame) -> pd.DataFrame:
    return pd.merge(employees, employee_uni, how='left', on='id')[['unique_id','name']]

In [25]:
print("\nPandas Result:")
print(replace_employee_id(Employees, EmployeeUNI))


Pandas Result:
   unique_id      name
0        NaN     Alice
1        NaN       Bob
2        2.0      Meir
3        3.0   Winston
4        1.0  Jonathan


### 7. Game Play Analysis IV

[Link to question](https://leetcode.com/problems/game-play-analysis-iv/)

This question is a bit more complicated. It first requires to find the first login for each player, and then find the total number of players that logged in the the exact day after the first day they logged in. This could be done with subquery or defining a second table, containing the first log of the players, and merging them.

In [26]:
# Example for 'Game Play Analysis IV'
data = {'player_id': [1, 1, 2, 3, 3],
        'device_id': [2, 2, 3, 1, 4],
        'event_date': ['2016-03-01', '2016-03-02', '2017-06-25', '2016-03-02', '2018-07-03'],
        'games_played': [5, 6, 1, 0, 5]}
Activity = pd.DataFrame(data)
print("Input DataFrame for Game Play Analysis:")
print(Activity)

Input DataFrame for Game Play Analysis:
   player_id  device_id  event_date  games_played
0          1          2  2016-03-01             5
1          1          2  2016-03-02             6
2          2          3  2017-06-25             1
3          3          1  2016-03-02             0
4          3          4  2018-07-03             5


SQL Solution:

In [27]:
# SQL Solution:
sql_query_game_play_analysis = """
WITH FirstLogin AS (
    SELECT player_id, MIN(event_date) AS first_login_date
    FROM Activity
    GROUP BY player_id
)
SELECT ROUND(CAST(COUNT(A.player_id) AS DECIMAL) / (SELECT COUNT(DISTINCT player_id) FROM Activity), 2) AS fraction
FROM Activity A
INNER JOIN FirstLogin F
ON A.player_id = F.player_id
WHERE A.event_date = DATE_ADD(F.first_login_date, INTERVAL 1 DAY);
"""

# OR Just using subquery by filtering the Activity table itself by the entries that have the first login date + 1 day

sql_query_game_play_analysis_alt = """
SELECT ROUND(CAST((
    SELECT COUNT(DISTINCT A.player_id)
    FROM Activity A
    WHERE(A.player_id, A.event_date) IN (
        SELECT player_id, DATE_ADD(MIN(event_date), INTERVAL 1 DAY)
        FROM Activity 
        GROUP BY player_id
        ))AS DECIMAL) 
    /(SELECT COUNT(DISTINCT player_id)
        FROM Activity)
    , 2) AS fraction
"""

Pandas Solution:

In [28]:
from datetime import timedelta

def gameplay_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    activity['event_date'] = pd.to_datetime(activity['event_date']) # Convert event_date to datetime
    first_login = activity.groupby('player_id')['event_date'].min().reset_index().rename(columns = {'event_date':'first_login'})
    first_login['target_date'] = first_login['first_login'] + timedelta(days=1)
    merged_df = pd.merge(activity,first_login, on='player_id', how='inner')
    filtered_df = merged_df[merged_df['event_date'] == merged_df['target_date']]
    return pd.DataFrame({"fraction": [round(filtered_df['player_id'].nunique()/activity['player_id'].nunique(), 2)]})

In [29]:
print("\nPandas Result:")
print(gameplay_analysis(Activity))


Pandas Result:
   fraction
0      0.33


### 8. Project Employees I

[Link to question](https://leetcode.com/problems/project-employees-i/)

The goal is to merge the project table and employee table using the employee_id and compute the average years of experience of the employees working on each project.

In [30]:
# Example for 'Project Employees I'
data_project = {'project_id': [1, 1, 1, 2, 2],
        'employee_id': [1, 2, 3, 1, 4]}
Project = pd.DataFrame(data_project)
data_employee = {'employee_id': [1, 2, 3, 4],
        'name': ['Khaled', 'Ali', 'John', 'Doe'],
        'experience_years': [3, 2, 1, 2]}
Employee = pd.DataFrame(data_employee)
print("Input DataFrame for Project:")
print(Project)
print("Input DataFrame for Employee:")
print(Employee)

Input DataFrame for Project:
   project_id  employee_id
0           1            1
1           1            2
2           1            3
3           2            1
4           2            4
Input DataFrame for Employee:
   employee_id    name  experience_years
0            1  Khaled                 3
1            2     Ali                 2
2            3    John                 1
3            4     Doe                 2


SQL Solution:

In [31]:
# SQL Solution:
sql_query_project_employees = """
SELECT P.project_id, ROUND(AVG(E.experience_years), 2) AS average_years
FROM Project P
INNER JOIN Employee E
ON P.employee_id=E.employee_id
GROUP BY P.project_id
"""

Pandas Solution:

In [32]:
def project_employees_i(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    return pd.merge(project,employee,how='inner',on='employee_id').groupby('project_id', as_index=False)['experience_years'].mean().rename(columns = {'experience_years':'average_years'}).assign(average_years=lambda df: df['average_years'].round(2))

In [33]:
print("\nPandas Result:")
print(project_employees_i(Project, Employee))


Pandas Result:
   project_id  average_years
0           1            2.0
1           2            2.5


### 9. Department Top Three Salaries

[Link to question](https://leetcode.com/problems/department-top-three-salaries/)

The goal is to rank the salaries for the employees in each department, and return only the top 3 ranked in each department.

In [34]:
# Example for 'Department Top Three Salaries'
data_employees = {'id': [1, 2, 3, 4, 5, 6, 7],
                  'name': ['Joe', 'Henry', 'Sam', 'Max', 'Janet', 'Randy', 'Will'],
                  'salary': [85000, 80000, 60000, 90000, 69000, 85000, 70000],
                  'departmentId': [1, 2, 2, 1, 1, 1, 1]}
Employees = pd.DataFrame(data_employees)
data_departments = {'id': [1, 2],
                    'name': ['IT', 'Sales']}
Departments = pd.DataFrame(data_departments)
print("Input DataFrame for Employees:")
print(Employees)
print("Input DataFrame for Departments:")
print(Departments)

Input DataFrame for Employees:
   id   name  salary  departmentId
0   1    Joe   85000             1
1   2  Henry   80000             2
2   3    Sam   60000             2
3   4    Max   90000             1
4   5  Janet   69000             1
5   6  Randy   85000             1
6   7   Will   70000             1
Input DataFrame for Departments:
   id   name
0   1     IT
1   2  Sales


SQL Solution:

In [35]:
# SQL Solution:
sql_query_top_three_salaries = """
WITH RankedEmployees AS (
    SELECT E.departmentId, E.name AS Employee, E.salary AS Salary, DENSE_RANK() OVER (PARTITION BY E.departmentId 
        ORDER BY E.salary DESC) AS salary_rank
    FROM Employee E
)

SELECT D.name AS Department, R.Employee, R.Salary
FROM RankedEmployees R
INNER JOIN Department D
ON R.departmentId = D.id
WHERE R.salary_rank<=3
"""

Pandas Solution:

In [36]:
def top_three_salaries(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    employee = employee.rename(columns={'name':'Employee','salary':'Salary'}).merge(department.rename(columns={'name':'Department'}), left_on = 'departmentId', right_on = 'id', how = 'inner').sort_values('Salary', ascending=False)
    employee['salary_rank'] = employee.groupby('Department')['Salary'].transform(lambda s: s.rank(method="dense", ascending=False))
    
    return employee.loc[employee['salary_rank']<=3, ['Department','Employee','Salary']]

In [37]:
print("\nPandas Result:")
print(top_three_salaries(Employees, Departments))


Pandas Result:
  Department Employee  Salary
3         IT      Max   90000
0         IT      Joe   85000
5         IT    Randy   85000
1      Sales    Henry   80000
6         IT     Will   70000
2      Sales      Sam   60000
